In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from reservoirpy import ESN

In [2]:
df = pd.read_csv("SWaT.csv")
df.columns = df.columns.str.strip()

df["Label"] = df["Normal/Attack"].apply(lambda x: 0 if x.strip().lower() == "normal" else 1)

feature_cols = df.columns[1:-2]  # between Timestamp and Normal/Attack/Label
X = df[feature_cols]
y = df["Label"].values

n = len(X)
train_end = int(0.6 * n)
val_end = int(0.8 * n)

X_train, X_val, X_test = X.iloc[:train_end], X.iloc[train_end:val_end], X.iloc[val_end:]
y_train, y_val, y_test = y[:train_end], y[train_end:val_end], y[val_end:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Train:", X_train_scaled.shape)
print("Val:", X_val_scaled.shape)
print("Test:", X_test_scaled.shape)

Train: (269951, 51)
Val: (89984, 51)
Test: (89984, 51)


In [3]:
base_units = 40
base_sr = 0.29
base_input_scaling = 1.0

Xtr = X_train_scaled
Xva = X_val_scaled
Xte = X_test_scaled

ytr = np.array(y_train).reshape(-1, 1)
yva = np.array(y_val)
yte = np.array(y_test)

esn_all = ESN(
    units=base_units,
    sr=base_sr,
    input_scaling=base_input_scaling,
)

esn_all = esn_all.fit(Xtr, ytr)

ytr_pred_raw = esn_all.run(Xtr)
yva_pred_raw = esn_all.run(Xva)
yte_pred_raw = esn_all.run(Xte)

ytr_pred = (ytr_pred_raw > 0.5).astype(int).ravel()
yva_pred = (yva_pred_raw > 0.5).astype(int).ravel()
yte_pred = (yte_pred_raw > 0.5).astype(int).ravel()

In [4]:
def compute_metrics(y_true, y_pred, split_name):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    print(f"{split_name} - Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")
    print(f"{split_name} confusion matrix:\n{cm}\n")
    return {
        "split": split_name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "tn": cm[0,0],
        "fp": cm[0,1],
        "fn": cm[1,0],
        "tp": cm[1,1],
    }

metrics_all = []
metrics_all.append(compute_metrics(y_train, ytr_pred, "train_all"))
metrics_all.append(compute_metrics(y_val,   yva_pred, "val_all"))
metrics_all.append(compute_metrics(y_test,  yte_pred, "test_all"))

metrics_all_df = pd.DataFrame(metrics_all)
metrics_all_df.to_csv("esn_allfeat_metrics.csv", index=False)
metrics_all_df

train_all - Acc: 0.9583, Prec: 0.9600, Rec: 0.9583, F1: 0.9561
train_all confusion matrix:
[[222087    207]
 [ 11054  36603]]

val_all - Acc: 0.9865, Prec: 0.9859, Rec: 0.9865, F1: 0.9845
val_all confusion matrix:
[[87645    79]
 [ 1135  1125]]

test_all - Acc: 0.9515, Prec: 0.9405, Rec: 0.9515, F1: 0.9352
test_all confusion matrix:
[[85019   261]
 [ 4106   598]]



,split,accuracy,precision,recall,f1,tn,fp,fn,tp
0,train_all,0.958285,0.959964,0.958285,0.956103,222087,207,11054,36603
1,val_all,0.986509,0.985889,0.986509,0.984493,87645,79,1135,1125
2,test_all,0.951469,0.940455,0.951469,0.935232,85019,261,4106,598
